In [1]:
#from bs4 import BeautifulSoup 
#bs4 is bad for some Nasdaq.com, since the request was never ended. It seems the server blocked the request by design.
#Thus, I use the Web Scraper (a Chrome plugin) to load stock news from Nasdaq.com, then export it as getNasdaqNews.csv
import pandas as pd
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\zhouy\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
df=pd.read_csv('getNasdaqNews.csv')
df

,scraper-Order,Title,Date
0,470,2 Reasons to Invest in Crypto -- And 1 Reason ...,18-Feb-22
1,471,Wall Street ends lower as investors eye Ukrain...,18-Feb-22
2,472,Rivian Stock Is Too Discounted to Ignore,18-Feb-22
3,473,Wall Street ends lower as investors eye Ukrain...,18-Feb-22
4,474,Wall St drops as Ukraine tensions sap risk app...,18-Feb-22
...,...,...,...
9970,10440,Alibaba Stock: What's Happening With China's A...,04-Sep-20
9971,10441,Amazon Japan submits improvement plan to regul...,04-Sep-20
9972,10442,Japanese shares fall on Wall Street tech rout,03-Sep-20
9973,10443,Amazon Japan submits improvement plan to regul...,03-Sep-20


In [8]:
vader=SentimentIntensityAnalyzer()
print(vader.polarity_scores("I don't think Apple is good compay. I think they will do poorly"))

{'neg': 0.0, 'neu': 0.775, 'pos': 0.225, 'compound': 0.4404}


In [9]:
print(vader.polarity_scores("I think Apple is bad compay. I think they will fail"))

{'neg': 0.5, 'neu': 0.5, 'pos': 0.0, 'compound': -0.7906}


In [10]:
print(vader.polarity_scores("I think Apple is great company. I think they will up to the moon"))

{'neg': 0.0, 'neu': 0.728, 'pos': 0.272, 'compound': 0.6249}


In [12]:
f=lambda title: vader.polarity_scores(title)['compound']
df['Compound']=df['Title'].apply(f)

In [54]:
df.dtypes

scraper-Order      int64
Title             object
Date              object
Compound         float64
dtype: object

In [63]:
df['Date'] = pd.to_datetime(df['Date'], format='%d-%b-%y')
df.dtypes

scraper-Order             int64
Title                    object
Date             datetime64[ns]
Compound                float64
dtype: object

In [64]:
df

,scraper-Order,Title,Date,Compound
0,470,2 Reasons to Invest in Crypto -- And 1 Reason ...,2022-02-18,0.0000
1,471,Wall Street ends lower as investors eye Ukrain...,2022-02-18,-0.5423
2,472,Rivian Stock Is Too Discounted to Ignore,2022-02-18,-0.3182
3,473,Wall Street ends lower as investors eye Ukrain...,2022-02-18,-0.5423
4,474,Wall St drops as Ukraine tensions sap risk app...,2022-02-18,-0.5859
...,...,...,...,...
9970,10440,Alibaba Stock: What's Happening With China's A...,2020-09-04,0.1779
9971,10441,Amazon Japan submits improvement plan to regul...,2020-09-04,0.5719
9972,10442,Japanese shares fall on Wall Street tech rout,2020-09-03,0.2960
9973,10443,Amazon Japan submits improvement plan to regul...,2020-09-03,0.5719


In [65]:
df_g=df.groupby('Date').agg(
    Day1ScoreMax =pd.NamedAgg(column='Compound', aggfunc='max'),
    Day1ScoreMin =pd.NamedAgg(column='Compound', aggfunc='min'),
    Day1ScoreMean=pd.NamedAgg(column='Compound', aggfunc='mean'),
    Day1ScoreSum =pd.NamedAgg(column='Compound', aggfunc='sum'),
    Day1CountAll =pd.NamedAgg(column='Compound', aggfunc='count'))
df_g

,Day1ScoreMax,Day1ScoreMin,Day1ScoreMean,Day1ScoreSum,Day1CountAll
Date,,,,,
2020-09-03,0.5719,0.2960,0.409700,1.2291,3
2020-09-04,0.6369,-0.6249,0.040576,1.3390,33
2020-09-05,0.4019,-0.5719,-0.042814,-0.2997,7
2020-09-06,0.6597,-0.4939,0.091386,0.6397,7
2020-09-07,0.6705,-0.6705,0.147060,0.7353,5
...,...,...,...,...,...
2022-02-14,0.6808,-0.6705,0.055271,1.1607,21
2022-02-15,0.6369,-0.4019,0.000641,0.0186,29
2022-02-16,0.7548,-0.5106,0.107104,2.7847,26


In [66]:
#get Day1CountPos
df_1=pd.DataFrame(df.groupby('Date').apply(lambda x: x[x['Compound'] >0]['Date'].count()))
df_1.rename(columns = {0:'Day1CountPos'}, inplace = True)
df_g=pd.merge(df_g, df_1, left_index=True, right_index=True) #default is inner join
df_g

,Day1ScoreMax,Day1ScoreMin,Day1ScoreMean,Day1ScoreSum,Day1CountAll,Day1CountPos
Date,,,,,,
2020-09-03,0.5719,0.2960,0.409700,1.2291,3,3
2020-09-04,0.6369,-0.6249,0.040576,1.3390,33,13
2020-09-05,0.4019,-0.5719,-0.042814,-0.2997,7,3
2020-09-06,0.6597,-0.4939,0.091386,0.6397,7,3
2020-09-07,0.6705,-0.6705,0.147060,0.7353,5,3
...,...,...,...,...,...,...
2022-02-14,0.6808,-0.6705,0.055271,1.1607,21,6
2022-02-15,0.6369,-0.4019,0.000641,0.0186,29,6
2022-02-16,0.7548,-0.5106,0.107104,2.7847,26,11


In [67]:
#get Day1CountNeg
df_1=pd.DataFrame(df.groupby('Date').apply(lambda x: x[x['Compound'] <0]['Date'].count()))
df_1.rename(columns = {0:'Day1CountNeg'}, inplace = True)
df_g=pd.merge(df_g, df_1, left_index=True, right_index=True) #default is inner join
df_g

,Day1ScoreMax,Day1ScoreMin,Day1ScoreMean,Day1ScoreSum,Day1CountAll,Day1CountPos,Day1CountNeg
Date,,,,,,,
2020-09-03,0.5719,0.2960,0.409700,1.2291,3,3,0
2020-09-04,0.6369,-0.6249,0.040576,1.3390,33,13,8
2020-09-05,0.4019,-0.5719,-0.042814,-0.2997,7,3,2
2020-09-06,0.6597,-0.4939,0.091386,0.6397,7,3,1
2020-09-07,0.6705,-0.6705,0.147060,0.7353,5,3,1
...,...,...,...,...,...,...,...
2022-02-14,0.6808,-0.6705,0.055271,1.1607,21,6,5
2022-02-15,0.6369,-0.4019,0.000641,0.0186,29,6,7
2022-02-16,0.7548,-0.5106,0.107104,2.7847,26,11,6


Date
2020-09-03     NaN
2020-09-04     NaN
2020-09-05    43.0
2020-09-06    47.0
2020-09-07    19.0
              ... 
2022-02-14    30.0
2022-02-15    55.0
2022-02-16    76.0
2022-02-17    77.0
2022-02-18    67.0
Name: Day1CountAll, Length: 534, dtype: float64

In [71]:
df_g['Day1CountPosPer']=df_g['Day1CountPos']/df_g['Day1CountAll']
df_g['Day1CountNegPer']=df_g['Day1CountNeg']/df_g['Day1CountAll']
df_g

,Day1ScoreMax,Day1ScoreMin,Day1ScoreMean,Day1ScoreSum,Day1CountAll,Day1CountPos,Day1CountNeg,Day1CountPosPer,Day1CountNegPer
Date,,,,,,,,,
2020-09-03,0.5719,0.2960,0.409700,1.2291,3,3,0,1.000000,0.000000
2020-09-04,0.6369,-0.6249,0.040576,1.3390,33,13,8,0.393939,0.242424
2020-09-05,0.4019,-0.5719,-0.042814,-0.2997,7,3,2,0.428571,0.285714
2020-09-06,0.6597,-0.4939,0.091386,0.6397,7,3,1,0.428571,0.142857
2020-09-07,0.6705,-0.6705,0.147060,0.7353,5,3,1,0.600000,0.200000
...,...,...,...,...,...,...,...,...,...
2022-02-14,0.6808,-0.6705,0.055271,1.1607,21,6,5,0.285714,0.238095
2022-02-15,0.6369,-0.4019,0.000641,0.0186,29,6,7,0.206897,0.241379
2022-02-16,0.7548,-0.5106,0.107104,2.7847,26,11,6,0.423077,0.230769


In [74]:
df_g['Day3ScoreMax'] =df_g['Day1ScoreMax'].rolling(3).max()
df_g['Day3ScoreMin'] =df_g['Day1ScoreMin'].rolling(3).min()
df_g['Day3ScoreMean']=df_g['Day1ScoreSum'].rolling(3).sum()/df_g['Day1CountAll'].rolling(3).sum()
df_g['Day3CountAll'] =df_g['Day1CountAll'].rolling(3).sum()
df_g['Day3CountPos'] =df_g['Day1CountPos'].rolling(3).sum()
df_g['Day3CountNeg'] =df_g['Day1CountNeg'].rolling(3).sum()
df_g['Day3CountPosPer']=df_g['Day3CountPos']/df_g['Day3CountAll']
df_g['Day3CountNegPer']=df_g['Day3CountNeg']/df_g['Day3CountAll']
df_g

,Day1ScoreMax,Day1ScoreMin,Day1ScoreMean,Day1ScoreSum,Day1CountAll,Day1CountPos,Day1CountNeg,Day1CountPosPer,Day1CountNegPer,Day3ScoreMax,Day3ScoreMin,Day3ScoreMean,Day3CountAll,Day3CountPos,Day3CountNeg,Day3CountPosPer,Day3CountNegPer
Date,,,,,,,,,,,,,,,,,
2020-09-03,0.5719,0.2960,0.409700,1.2291,3,3,0,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-04,0.6369,-0.6249,0.040576,1.3390,33,13,8,0.393939,0.242424,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-09-05,0.4019,-0.5719,-0.042814,-0.2997,7,3,2,0.428571,0.285714,0.6369,-0.6249,0.052753,43.0,19.0,10.0,0.441860,0.232558
2020-09-06,0.6597,-0.4939,0.091386,0.6397,7,3,1,0.428571,0.142857,0.6597,-0.6249,0.035723,47.0,19.0,11.0,0.404255,0.234043
2020-09-07,0.6705,-0.6705,0.147060,0.7353,5,3,1,0.600000,0.200000,0.6705,-0.6705,0.056595,19.0,9.0,4.0,0.473684,0.210526
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-14,0.6808,-0.6705,0.055271,1.1607,21,6,5,0.285714,0.238095,0.6808,-0.6705,0.081123,30.0,11.0,6.0,0.366667,0.200000
2022-02-15,0.6369,-0.4019,0.000641,0.0186,29,6,7,0.206897,0.241379,0.6808,-0.6705,0.050485,55.0,16.0,12.0,0.290909,0.218182
2022-02-16,0.7548,-0.5106,0.107104,2.7847,26,11,6,0.423077,0.230769,0.7548,-0.6705,0.052158,76.0,23.0,18.0,0.302632,0.236842


In [77]:
df_g=df_g.dropna()
df_g

,Day1ScoreMax,Day1ScoreMin,Day1ScoreMean,Day1ScoreSum,Day1CountAll,Day1CountPos,Day1CountNeg,Day1CountPosPer,Day1CountNegPer,Day3ScoreMax,Day3ScoreMin,Day3ScoreMean,Day3CountAll,Day3CountPos,Day3CountNeg,Day3CountPosPer,Day3CountNegPer
Date,,,,,,,,,,,,,,,,,
2020-09-05,0.4019,-0.5719,-0.042814,-0.2997,7,3,2,0.428571,0.285714,0.6369,-0.6249,0.052753,43.0,19.0,10.0,0.441860,0.232558
2020-09-06,0.6597,-0.4939,0.091386,0.6397,7,3,1,0.428571,0.142857,0.6597,-0.6249,0.035723,47.0,19.0,11.0,0.404255,0.234043
2020-09-07,0.6705,-0.6705,0.147060,0.7353,5,3,1,0.600000,0.200000,0.6705,-0.6705,0.056595,19.0,9.0,4.0,0.473684,0.210526
2020-09-08,0.6705,-0.6808,-0.029977,-0.9293,31,7,11,0.225806,0.354839,0.6705,-0.6808,0.010365,43.0,13.0,13.0,0.302326,0.302326
2020-09-09,0.6369,-0.6597,0.073768,2.5081,34,10,3,0.294118,0.088235,0.6705,-0.6808,0.033059,70.0,20.0,15.0,0.285714,0.214286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-14,0.6808,-0.6705,0.055271,1.1607,21,6,5,0.285714,0.238095,0.6808,-0.6705,0.081123,30.0,11.0,6.0,0.366667,0.200000
2022-02-15,0.6369,-0.4019,0.000641,0.0186,29,6,7,0.206897,0.241379,0.6808,-0.6705,0.050485,55.0,16.0,12.0,0.290909,0.218182
2022-02-16,0.7548,-0.5106,0.107104,2.7847,26,11,6,0.423077,0.230769,0.7548,-0.6705,0.052158,76.0,23.0,18.0,0.302632,0.236842


In [78]:
df_g=df_g.drop(columns=['Day1ScoreSum','Day1CountPos','Day1CountNeg','Day3CountPos','Day3CountNeg'],axis=1)
df_g

,Day1ScoreMax,Day1ScoreMin,Day1ScoreMean,Day1CountAll,Day1CountPosPer,Day1CountNegPer,Day3ScoreMax,Day3ScoreMin,Day3ScoreMean,Day3CountAll,Day3CountPosPer,Day3CountNegPer
Date,,,,,,,,,,,,
2020-09-05,0.4019,-0.5719,-0.042814,7,0.428571,0.285714,0.6369,-0.6249,0.052753,43.0,0.441860,0.232558
2020-09-06,0.6597,-0.4939,0.091386,7,0.428571,0.142857,0.6597,-0.6249,0.035723,47.0,0.404255,0.234043
2020-09-07,0.6705,-0.6705,0.147060,5,0.600000,0.200000,0.6705,-0.6705,0.056595,19.0,0.473684,0.210526
2020-09-08,0.6705,-0.6808,-0.029977,31,0.225806,0.354839,0.6705,-0.6808,0.010365,43.0,0.302326,0.302326
2020-09-09,0.6369,-0.6597,0.073768,34,0.294118,0.088235,0.6705,-0.6808,0.033059,70.0,0.285714,0.214286
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-14,0.6808,-0.6705,0.055271,21,0.285714,0.238095,0.6808,-0.6705,0.081123,30.0,0.366667,0.200000
2022-02-15,0.6369,-0.4019,0.000641,29,0.206897,0.241379,0.6808,-0.6705,0.050485,55.0,0.290909,0.218182
2022-02-16,0.7548,-0.5106,0.107104,26,0.423077,0.230769,0.7548,-0.6705,0.052158,76.0,0.302632,0.236842


In [79]:
df_g[['Day1CountAll','Day3CountAll']].max()

Day1CountAll     59.0
Day3CountAll    139.0
dtype: float64

In [80]:
df_g['Day1CountAll']=df_g['Day1CountAll']/59
df_g['Day3CountAll']=df_g['Day3CountAll']/139
df_g

,Day1ScoreMax,Day1ScoreMin,Day1ScoreMean,Day1CountAll,Day1CountPosPer,Day1CountNegPer,Day3ScoreMax,Day3ScoreMin,Day3ScoreMean,Day3CountAll,Day3CountPosPer,Day3CountNegPer
Date,,,,,,,,,,,,
2020-09-05,0.4019,-0.5719,-0.042814,0.118644,0.428571,0.285714,0.6369,-0.6249,0.052753,0.309353,0.441860,0.232558
2020-09-06,0.6597,-0.4939,0.091386,0.118644,0.428571,0.142857,0.6597,-0.6249,0.035723,0.338129,0.404255,0.234043
2020-09-07,0.6705,-0.6705,0.147060,0.084746,0.600000,0.200000,0.6705,-0.6705,0.056595,0.136691,0.473684,0.210526
2020-09-08,0.6705,-0.6808,-0.029977,0.525424,0.225806,0.354839,0.6705,-0.6808,0.010365,0.309353,0.302326,0.302326
2020-09-09,0.6369,-0.6597,0.073768,0.576271,0.294118,0.088235,0.6705,-0.6808,0.033059,0.503597,0.285714,0.214286
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-02-14,0.6808,-0.6705,0.055271,0.355932,0.285714,0.238095,0.6808,-0.6705,0.081123,0.215827,0.366667,0.200000
2022-02-15,0.6369,-0.4019,0.000641,0.491525,0.206897,0.241379,0.6808,-0.6705,0.050485,0.395683,0.290909,0.218182
2022-02-16,0.7548,-0.5106,0.107104,0.440678,0.423077,0.230769,0.7548,-0.6705,0.052158,0.546763,0.302632,0.236842


In [82]:
df_g.to_csv('getNasdaqNews_preprocess.csv')